<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/EventProfiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QSTK

    100% |████████████████████████████████| 42.4MB 519kB/s 
    100% |████████████████████████████████| 235kB 20.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/18/00/6b/01cf34934bf6a54b9d6df1c7a40230e18be1fd05d28e97d5ef
  Stored in directory: /root/.cache/pip/wheels/3e/57/ae/3a3f171dc8c46ac987fa8d7a56fcc19d82ff1a7151cf46a228
Successfully built QSTK python-dateutil
fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.0.0 which is incompatible.
botocore 1.12.88 has requirement python-dateutil<3.0.0,>=2.1; python_version >= "2.7", but you'll have python-dateutil 1.5 which is incompatible.
bokeh 1.0.4 has requirement python-dateutil>=2.1, but you'll have python-dateutil 1.5 which is incompatible.
matplotlib 2.2.3 has requirement python-dateutil>=2.1, but you'll have python-dateutil 1.5 which is incompatible.
jupyter-client 5.2.4 has requirement python-dateutil>=2.1, but you'll have python-dateutil 1.5 which is incompatible.
  Found existing installation: python-dateutil 2

In [1]:
!pip install pandas_datareader

<h3> Fetching all tickers from sp500</h2>

In [2]:
def get_sp500_tickers():
  import bs4 as bs    
  import pickle    
  import requests    
  import lxml    
  resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')        
  soup = bs.BeautifulSoup(resp.text,'lxml')        
  table = soup.find('table', {'class': 'wikitable sortable'})        

  tickers = []

  for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[1].text
    tickers.append(str(ticker))

  return tickers    

res = get_sp500_tickers()
res

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'APC',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BHGE',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BBT',
 'BDX',
 'BRK-B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BHF',
 'BMY',
 'AVGO',
 'BR',
 'BF-B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CPRI',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CBS',
 'CE',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS

<h2>Writing code to fetch data from yahoo (use IEX for recent data) </h2>

In [0]:
def get_data_from_yahoo(start, end, ticker, source='iex'):
  
  import pandas_datareader.data as dr
  import numpy as np
  from datetime import datetime
  try:
    return dr.DataReader(ticker, source, start, end)
  except Exception as e:
    print 'Exception when retrieving data for %s' % ticker



<h2> Importing required lbraries.(This code is borrowed from Computational Investing. course code only works for past data)</h2>

In [0]:


import pandas as pd; pd.TimeSeries = pd.Series


import numpy as np
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import datetime as dt
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkstudy.EventProfiler as ep


<h2> Importing functions from tutorial3 to reuse event profiler </h2>

In [0]:
def find_events(ls_symbols, d_data):
    ''' Finding the event dataframe '''
    df_close = d_data['close']
    #ts_market = df_close['SPY']
    # Got it. we need to replace this with pandas.datareader
    print "Finding Events"

    # Creating an empty dataframe
    df_events = copy.deepcopy(df_close)
    df_events = df_events * np.NAN

    # Time stamps for the event range
    ldt_timestamps = df_close.index


    print 'Loopingthrough all symbols....'
    print df_close
    for s_sym in ls_symbols:
        try:
          for i in range(1, len(ldt_timestamps)):
              # Calculating the returns for this timestamp
              f_symprice_today = df_close[s_sym].ix[ldt_timestamps[i]]
              print (f_symprice_today)
              f_symprice_yest = df_close[s_sym].ix[ldt_timestamps[i - 1]]
              f_marketprice_today = ts_market.ix[ldt_timestamps[i]]
              f_marketprice_yest = ts_market.ix[ldt_timestamps[i - 1]]
              f_symreturn_today = (f_symprice_today / f_symprice_yest) - 1
              f_marketreturn_today = (f_marketprice_today / f_marketprice_yest) - 1

              # Event is found if the symbol is down more then 3% while the
              # market is up more then 2%
              #if f_symreturn_today <= -0.03 and f_marketreturn_today >= 0.02:
              # checking impact of 10% drop
              if f_symprice_today <= f_symprice_yest * 0.9:
                  print 'Found event for  {}'.format(s_sym)
                  df_events[s_sym].ix[ldt_timestamps[i]] = 1
        except Exception as e:
            print 'Exception %s ' % str(e)
          

    return df_events


In [0]:
def kick_off_events(start_date, end_date):
    ls_symbols = get_sp500_tickers()[0:5]
    ls_symbols.append('SPY')
    print 'About to get data for {}'.format(','.join(ls_symbols))
    ls_keys = ['close', 'volume']
    ldf_data = [get_data_from_yahoo(start_date, end_date, ticker) for ticker in ls_symbols]
    from pprint import pprint
    d_data = dict(zip(ls_keys, ldf_data))

    for s_key in ls_keys:
        try: 
          d_data[s_key] = d_data[s_key].fillna(method='ffill')
          d_data[s_key] = d_data[s_key].fillna(method='bfill')
          d_data[s_key] = d_data[s_key].fillna(1.0)
        except Exception as e:
          print 'Exception in dealing with %s data' % s_key

    print 'Finding event snow.....'
    df_events = find_events(ls_symbols, d_data)
    print "Creating Study for {}".format(len(df_events))
    print df_events.info()
    df_events.to_csv('EventProfile%s.csv')
    ep.eventprofiler(df_events, d_data, i_lookback=20, i_lookforward=20,
                s_filename='MyEventStudy-%s.pdf' , b_market_neutral=True, b_errorbars=True,
                s_market_sym='SPY')

  
  
  
  

In [23]:
from datetime import date
res = get_data_from_yahoo(date(2019,1,10), date(2019,1,30), 'SPY')
res

kick_off_events(date(2018,1,1), date(2018,12,31))


About to get data for MMM,ABT,ABBV,ABMD,ACN,SPY
Finding event snow.....
Finding Events
Loopingthrough all symbols....
                open      high       low     close   volume
date                                                       
2018-01-02  229.6870  230.9436  226.7888  229.5506  2930951
2018-01-03  228.9953  229.6383  227.2613  229.5408  2193713
2018-01-04  230.8754  233.2524  230.3591  232.5413  2243101
2018-01-05  232.4828  234.6747  231.5963  234.3532  1835909
2018-01-08  233.1939  234.7136  232.9991  233.5933  1869025
2018-01-09  233.4083  235.5363  233.1550  235.0448  1695114
2018-01-10  234.7721  236.3015  233.8271  234.9085  1640853
2018-01-11  234.5188  236.0774  233.5544  236.0482  1487650
2018-01-12  236.7886  239.6429  236.2333  238.1524  1974312
2018-01-16  238.9609  240.8021  237.5289  238.4154  2411131
2018-01-17  240.4709  242.1075  239.4480  241.6983  2519000
2018-01-18  241.7178  242.5653  238.5908  239.5552  2233505
2018-01-19  240.2566  242.0783  239.2532  

KeyError: ignored